In [ ]:
# import modules and define functions
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, ticker, cm
import matplotlib as mpl
import os
os.environ['OMP_NUM_THREADS'] = "1"
def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

In [ ]:
outfs_dimer = [i for i in os.listdir(os.getcwd()) if i.endswith('.out') and 'he2' in i]
outfs_mononer = [i for i in os.listdir(os.getcwd()) if i.endswith('.out') and 'he_' in i]
data_dimer_dict = {}
data_monomer_dict = {}
for outf in outfs_dimer:
    method = outf.split('_')[3].upper()
    lines = open(outf, 'r').read().splitlines()
    for i, line in enumerate(lines):
        if "The Calculated Surface using the 'Actual Energy'" in line:
            data = lines[i+1:i+1+21]
            data = [list(filter(None, i.split(' '))) for i in data]
            keys, es = np.array(data, dtype=float).T
            break
    data_dimer_dict[method] = es

for outf in outfs_mononer:
    method = outf.split('_')[3].upper()
    lines = open(outf, 'r').read().splitlines()
    for i, line in enumerate(lines):
        if 'FINAL SINGLE' in line:
            e = float(list(filter(None, line.split(' ')))[4])
            break
    data_monomer_dict[method] = e

common_keys = list(set(data_dimer_dict.keys()) & set(data_monomer_dict.keys()))
data_dict = {}
for key in common_keys:
    data_dict[key] = data_dimer_dict[key] - np.array(data_monomer_dict[key]) * 2
    

In [ ]:
df = pd.DataFrame(data_dict, dtype=float).T
df.columns = keys
df = df.T
df = df[['CCSD-T', 'HF', 'B3LYP', 'PBE0', 'M06', 'CAM-B3LYP', 'LC-BLYP', 'LC-PBE', 'WB97X']]

In [ ]:
# plot the change of energy 
xlabel = 'log(He-He distance) / A'
ylabel = 'log(absolute binding energy) / Eh'
title = 'Dispersion interaction'
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

# fitting
a, b = np.polyfit(np.log10(df.index), np.log10(np.abs(df['CCSD-T'].values)), 1)

# plotting
fig, ax = plt.subplots(figsize=(7,6))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
ax.yaxis.set_major_locator(ticker.MultipleLocator(2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(1))
ax.set_ylim(-9, -4)
ax.set_xlim(0.69,1.01)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
ax.set_title(title, fontsize=22)
text = 'y = %.4fx + %.4f' %(a, b)
ax.text(0.85, -4.5, text, fontsize=12, bbox=dict(facecolor='red', alpha=0.7))
ax.plot(np.log10(df.index), np.log10(np.abs(df.values)), label=df.columns)
ax.plot(np.log10(df.index), np.log10(df.index)*a+b, 'r--')

ax.legend()
